In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from scipy.interpolate import splrep, splev
import datetime as dt
import matplotlib.pyplot as plt
from fbprophet import Prophet
from scipy.interpolate import InterpolatedUnivariateSpline
%matplotlib inline
plt.rcParams.update({'font.size': 18})

ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.


In [2]:
def smape(satellite_predicted_values, satellite_true_values):
    # the division, addition and subtraction are pointwise
    return np.mean(np.abs((satellite_predicted_values - satellite_true_values)/
                (np.abs(satellite_predicted_values) + np.abs(satellite_true_values))))


def drop_close(t, x, eps=10**9):
    '''
    t = time array, x = data array, eps is in nanoseconds
    Returns entries in t,x with corresponding consecutive times > eps
    
    '''
    t = np.array(t) #if not already np array, convert
    x = np.array(x) #if t or x are pandas Series, will have dimension mismatch
    far = np.concatenate([(t[1:] - t[:-1]) > eps, [True]])
    return t[far], x[far]


def resample(t, x, step=10 * 10**9, t_new=None, return_t=False):
    '''
    t: time array (or series); 
    x: data array (or series); 
    t_new: new time scale from start to end of t with step size step;
    step: = 10 seconds by default; 
    return_t: by default, do not return resampled times
    
    resample time series or array by 10 (default) sec intervals and 
    return new time series (if t_new=True) and spline approximation series for data
    
    
    '''
    t, x = drop_close(t, x)
    if t_new is None:
        t_new = np.arange(t[0], t[-1], step)
    try:
        spl = splrep(t, x)
        x_new = splev(t_new, spl)
    except:
        raise ValueError(f'interpolation error, x length = {len(x)}, \
        t_new length = {len(t_new)}')

    return (t_new, x_new) if return_t else x_new


def get_peaks(array):
    '''
    returns index of "sharp" peaks, excluding first and last values of array
    
    index of "smooth peaks", e.g. 1 2 3 9 9 3 2 1, is not returned
    '''
    return np.where((array[1:-1] > array[2:]) & (array[1:-1] > array[:-2]))[0] + 1


def get_satellite_data(data, sat_id):
    '''
    returns all data for particular satellite by id
    '''
    return data[data['sat_id'] == sat_id]


def remove_time_jumps(satellite_data, features_list=('x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim'),
                      suffix='_jumps_removed',time_threshold = 0.00003):
    #time_threshold 0.00003 sufficient for test and train
    #time_threshold 0.00002 will throw errors
    '''
    removes time jumps in the simulation for a single satellite
    for train and test data, sufficient to set time_threshold at default
    '''
    satellite_data['t'] = ((pd.to_datetime(satellite_data['epoch']) - pd.to_datetime(satellite_data['epoch'])[0]) /
                           np.timedelta64(1, 'D')).astype(float)
    satellite_data['dt'] = satellite_data['t'].diff()
    # the most frequent time interval
    t_standard = satellite_data['dt'].value_counts().index[0]

    # time steps used for simulations
    satellite_data['t_sim'] = satellite_data.index.values * t_standard

    n = satellite_data.shape[0]
    corrected_features = []
    for feature_name in features_list:
        corrected_feature = [0] * n
        corrected_feature[0] = satellite_data[feature_name][0]

        feature = satellite_data[feature_name]

        for j in range(1, n - 1):
            if satellite_data.t[j] < satellite_data.t_sim[j] - time_threshold:
                # approximate by the left side
    
                # look for the interval
                step = 0
                while satellite_data.t[j] < satellite_data.t_sim[j - step] - time_threshold:
                    step += 1
                #             print(step)
                corrected_feature[j] = feature[j - step] - (satellite_data.t_sim[j - step] - satellite_data.t[j]) / (
                            satellite_data.t_sim[j - step] - satellite_data.t_sim[j - step - 1]) * (
                                              feature[j - step] - feature[j - step - 1])
            elif satellite_data.t[j] > satellite_data.t_sim[j] + time_threshold:
                # approximate by the right side
    
                # look for the interval
                step = 0
                while satellite_data.t[j] > satellite_data.t_sim[j + step] + time_threshold:
                    step += 1
    
                corrected_feature[j] = feature[j + step + 1] - (satellite_data.t_sim[j + step + 1] - satellite_data.t[j]) / (
                            satellite_data.t_sim[j + step + 1] - satellite_data.t_sim[j + step]) * (
                                              feature[j + step + 1] - feature[j + step])
            else:
                corrected_feature[j] = feature[j]
    
        corrected_feature[n - 1] = feature[n - 1] + corrected_feature[n - 2] - feature[n - 2]
        corrected_features.append(corrected_feature)
    return pd.DataFrame(corrected_features, index=[f + suffix for f in features_list]).T

In [3]:
def insert_previous_and_shift(df,col_name,ind):
    '''
    input a data frame (df), column name (col_name), and index (ind)
    insert previous value of df[col_name] at index and shift the rest 
    of df[col_name] from ind by +1;
    This is used for remove_time_jumps_fast
    '''
    shifted_series = df[col_name].shift(1)
    df[col_name].iloc[ind] = df[col_name].iloc[ind-1]
    df[col_name].iloc[ind+1:] = shifted_series.iloc[ind+1:]
    return df

In [4]:
def remove_time_jumps_fast(data, features_list=
                           ('x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim'),
                           threshold = 0.000001):
    #time_threshold 0.00003 sufficient for test and train
    #time_threshold 0.00002 will throw errors
    '''
    removes time jumps in the simulation for a single satellite
    for train and test data, sufficient to set time_threshold at default
    s_data = satellite data
    the features are replaced by the correction
    note that threshold here is not the same as in remove_time_jumps
    '''
    data['t'] = ((pd.to_datetime(data['epoch']) - pd.to_datetime(data['epoch'])[0]) /
                               np.timedelta64(1, 'D')).astype(float)
    data['dt'] = data['t'].diff(1)

    index_for_correction = data[data['dt'] < threshold].index 
    #print(index_for_correction)
    if list(index_for_correction): #if non empty
        for feature in features_list:
            for i in index_for_correction:
                data = insert_previous_and_shift(data,feature,i)
    return data

## Transforming all the data

In [9]:
import utils
from LinearAlignment import LinearAlignment

In [6]:
features_list=['x', 'y', 'z', 'Vx', 'Vy', 'Vz']

In [7]:
data = pd.read_csv('data/train.csv', index_col = 'id')
data['time'] = data['epoch']
data['epoch'] = pd.to_datetime(data['epoch']).values.astype(float)
data.head()

,epoch,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim,time
id,,,,,,,,,,,,,,,
0,1.388534e+18,0,-8855.823863,13117.780146,-20728.353233,-0.908303,-3.808436,-2.022083,-8843.131454,13138.221690,-20741.615306,-0.907527,-3.804930,-2.024133,2014-01-01T00:00:00.000
1,1.388537e+18,0,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468,2014-01-01T00:46:43.000
2,1.388540e+18,0,-10578.684043,-10180.467460,-24238.280949,0.277435,-4.047522,0.723155,-10571.858472,-10145.939908,-24271.169776,0.274880,-4.046788,0.718768,2014-01-01T01:33:26.001
3,1.388543e+18,0,-9148.251857,-20651.437460,-20720.381279,0.715600,-3.373762,1.722115,-9149.620794,-20618.200201,-20765.019094,0.712437,-3.375202,1.718306,2014-01-01T02:20:09.001
4,1.388546e+18,0,-6719.092336,-28929.061629,-14938.907967,0.992507,-2.519732,2.344703,-6729.358857,-28902.271436,-14992.399986,0.989382,-2.522618,2.342237,2014-01-01T03:06:52.002


In [8]:
test_data = pd.read_csv('data/test.csv', index_col = 'id')
test_data['time'] = test_data['epoch']
test_data['epoch'] = pd.to_datetime(test_data['epoch']).values.astype(float)

In [10]:
test_data.head()

,sat_id,epoch,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim,time
id,,,,,,,,,
3927,1,1.391213e+18,-13366.891347,-14236.753503,6386.774555,4.333815,-0.692764,0.810774,2014-02-01T00:01:45.162
3928,1,1.391214e+18,-7370.434039,-14498.771520,7130.411325,5.077413,0.360609,0.313402,2014-02-01T00:22:57.007
3929,1,1.391215e+18,-572.068654,-13065.289498,7033.794876,5.519106,2.012830,-0.539412,2014-02-01T00:44:08.852
3930,1,1.391217e+18,6208.945257,-9076.852425,5548.296900,4.849212,4.338955,-1.869600,2014-02-01T01:05:20.697
3931,1,1.391218e+18,10768.200284,-2199.706707,2272.014862,1.940505,6.192887,-3.167724,2014-02-01T01:26:32.542


In [223]:
result_df = []
alignment_model = LinearAlignment()

satellites_list = test_data['sat_id'].unique()

transf_df = pd.DataFrame([])
# for sat_id in tqdm(satellites_list):
for sat_id in tqdm(satellites_list):
    
#     sat_data = utils.get_satellite_data(data, sat_id)
    try:
        sat_data = get_satellite_data(data, sat_id)
        n_first = sat_data.shape[0]
        
        test_sat_data = get_satellite_data(test_data, sat_id)
        
        sat_data = pd.concat([sat_data,test_sat_data], axis = 0, sort = False)
                
        index = sat_data.index
        pred = pd.DataFrame(index = index )
        
        sat_data = remove_time_jumps_fast(sat_data.reset_index(drop=True))
        sat_data.set_index(index)
#         sat_data = get_satellite_data(data, sat_id) # run to check the smape score before any transformations
    except KeyError as e:
        print(f'jump removal failed for satellite {sat_id}:\t{type(e).__name__} {e}')
        continue
    
    
    n_train = 4*len(sat_data) // 10
    
    train_sat_data = sat_data[:n_train]
    
    
    pred['epoch'] = sat_data['epoch'].values
    pred['t'] = sat_data['t'].values
    pred['sat_id'] = sat_id
    
    if sat_id in set([372,587,523,473,514]):
        
        for feature_name in features_list:
            pred.loc[:,f'{feature_name}_sim']  = sat_data.loc[:, f'{feature_name}_sim'].values
            pred.loc[:,feature_name]=sat_data.loc[:, feature_name].values
    else:
        
        try:

            for feature_name in features_list:
                alignment_model.fit(t=train_sat_data['epoch'].values,
                                    x=-train_sat_data[f'{feature_name}_sim'].values,
                                    gt=-train_sat_data[feature_name].values)
                option1 = alignment_model.predict(t=sat_data['epoch'].values,
                                                             x=sat_data[f'{feature_name}_sim'].values)
                alignment_model.fit(t=train_sat_data['epoch'].values,
                                    x= train_sat_data[f'{feature_name}_sim'].values,
                                    gt= train_sat_data[feature_name].values)

                option2 = alignment_model.predict(t=sat_data['epoch'].values,
                                                             x=sat_data[f'{feature_name}_sim'].values)

                if utils.smape(option1[n_train:n_first],sat_data[feature_name].values[n_train:n_first]) < utils.smape(option2[n_train:n_first],sat_data[feature_name].values[n_train:n_first]):
                    alignment_model.fit(t=sat_data['epoch'].iloc[:n_first].values,
                                    x=-sat_data[f'{feature_name}_sim'].iloc[:n_first].values,
                                    gt=-sat_data[feature_name].iloc[:n_first].values)

                    pred[f'{feature_name}_sim'] = alignment_model.predict(t=sat_data['epoch'].values,
                                                             x=sat_data[f'{feature_name}_sim'].values)
                else:
                    alignment_model.fit(t=sat_data['epoch'].iloc[:n_first].values,
                                    x=sat_data[f'{feature_name}_sim'].iloc[:n_first].values,
                                    gt=sat_data[feature_name].iloc[:n_first].values)
                    pred.loc[:,f'{feature_name}_sim']  = alignment_model.predict(t=sat_data['epoch'].values,
                                                             x=sat_data[f'{feature_name}_sim'].values)

                pred.loc[:,feature_name]=sat_data.loc[:, feature_name].values



        except Exception as e:
            print(f'linear alignment failed for satellite {sat_id}:\t{type(e).__name__} {e}')
            continue
    
    transf_df = transf_df.append(pred, sort = False)


 45%|████████████████████████████████████                                            | 135/300 [01:48<01:54,  1.44it/s]


 90%|████████████████████████████████████████████████████████████████████████▎       | 271/300 [03:55<00:19,  1.49it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [04:26<00:00,  1.20s/it]

In [12]:
transf_df.head()

,epoch,t,sat_id,x_sim,x,y_sim,y,z_sim,z,Vx_sim,Vx,Vy_sim,Vy,Vz_sim,Vz
id,,,,,,,,,,,,,,,
1819,1.388534e+18,0.000000,1,10405.813755,10390.313089,-2771.180760,-2796.458271,3166.926302,3179.562085,2.508879,2.520477,6.152996,6.149930,-2.826227,-2.827599
1820,1.388536e+18,0.014720,1,11199.459274,11195.606833,5105.200658,5078.653968,-848.597754,-839.076593,-1.132222,-1.126667,5.822436,5.826412,-3.250054,-3.255872
1821,1.388537e+18,0.029441,1,8237.279843,8235.556436,11469.206013,11445.904263,-4684.829256,-4680.514023,-3.228498,-3.230227,4.124297,4.126433,-2.700287,-2.704441
1822,1.388538e+18,0.044161,1,3565.004307,3560.149776,15655.733441,15634.195146,-7655.277835,-7654.177182,-3.960607,-3.964696,2.521337,2.520867,-1.976770,-1.978151
1823,1.388539e+18,0.058882,1,-1571.839073,-1580.476891,18044.683022,18023.318335,-9755.479443,-9755.287599,-4.046854,-4.050865,1.298053,1.296388,-1.347061,-1.346512


In [13]:
transf_df.shape

(599254, 15)

In [ ]:
# # for now, simply append non-transformed values fot the satellites 252
# transf_df = transf_df.append(remove_time_jumps_fast(get_satellite_data(data, 252))[transf_df.columns])
# transf_df = transf_df.append(remove_time_jumps_fast(get_satellite_data(data, 301))[transf_df.columns])
# transf_df.shape

In [14]:
fail_list = []
satellites_list = transf_df['sat_id'].unique()
res = pd.DataFrame([])
for sat_id in tqdm(satellites_list[200:]):
    
    
    
    sat_data = transf_df[transf_df.sat_id == sat_id]
    
    n_train = 4* len(sat_data) // 10
    n_first = len(sat_data[~pd.isna(sat_data.x)])

    pred = pd.DataFrame(index = sat_data.iloc[n_first:].index)
    pred['epoch'] = sat_data['epoch'].iloc[n_first:].values
    pred['t'] = sat_data['t'].iloc[n_first:].values
    pred['sat_id'] = sat_id

    if sat_id in [26]:
        for feature_name in features_list:
            pred[feature_name] = sat_data[f'{feature_name}_sim'].values
    else:
        try:

            for feature_name in features_list:

                q = InterpolatedUnivariateSpline(sat_data['t'][:n_train], sat_data[f'{feature_name}_sim'][:n_train]).roots()
                T = (q[2]-q[0])

                # create a DataFrame for prophet model()
                ts = pd.DataFrame([])
                ts['ds'] = pd.to_datetime(sat_data ['epoch']).values
                ts['y'] = sat_data [feature_name].values-sat_data [f'{feature_name}_sim'].values

                # define the time frame for the first forecast 
                future =ts[n_train:n_first] [['ds']]

                # define the model
                m = Prophet(changepoint_prior_scale = T/20,
                weekly_seasonality = False,
                yearly_seasonality = False).add_seasonality(name='grid', period = T, fourier_order=50)

                #fit the model and make the forecast
                m.fit(ts[:n_train])
                forecast = m.predict(future)

                new_values = sat_data.loc[:,f'{feature_name}_sim'].iloc[n_train:n_first].values+forecast.loc[:, 'yhat'].values

                option1 = sat_data[f'{feature_name}_sim'].values


                option2 = np.concatenate((sat_data[f'{feature_name}_sim'].values[:n_train], new_values), axis=None)


                print('{}[{}]  no correction: {}, correction:{}'.format(sat_id,
                                                                    feature_name,
                                                                    utils.smape(option1[n_train:n_first],sat_data[feature_name].iloc[n_train:n_first].values),
                                                                    utils.smape(option2[n_train:n_first],sat_data[feature_name].iloc[n_train:n_first].values)))

                if utils.smape(option1[n_train:n_first],sat_data[feature_name].iloc[n_train:n_first].values) < utils.smape(option2[n_train:n_first],sat_data[feature_name].iloc[n_train:n_first].values):
                    pred[feature_name] = option1[n_first:]
                    print(sat_id, feature_name, 'does not work')
                    fail_list.append((sat_id, feature_name))
                else:
                    # fit a new model on the whole training range

                    # define the time frame for the first forecast 
                    future =ts[n_first:] [['ds']]

                    # define the model
                    m = Prophet(changepoint_prior_scale = T/20,
                    weekly_seasonality = False,
                    yearly_seasonality = False).add_seasonality(name='grid', period = T, fourier_order=50)

                    #fit the model and make the forecast
                    m.fit(ts[:n_first])
                    forecast = m.predict(future)

                    pred[feature_name]  = sat_data.loc[:,f'{feature_name}_sim'].iloc[n_first:].values+forecast.loc[:, 'yhat'].values





        except Exception as e:
            print(f'phophet failed for satellite {sat_id}:\t{type(e).__name__} {e}')
            continue
        
#     sat_sim_smape = utils.smape(sat_data.loc[n_train:, [f'{f}_sim' for f in features_list]].values,
#                                 sat_data.loc[n_train:, features_list].values)
#     sat_new_smape = utils.smape(pred.loc[n_train:, features_list].values,
#                                 sat_data.loc[n_train:, features_list].values)
#     result_df.append([sat_id, sat_sim_smape, sat_new_smape])
    res = res.append(pred, sort=False)

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\Tatiana\Anaconda3\envs\idao\lib\site-packages\pystan\misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


409[x]  no correction: 0.010941483528247553, correction:0.00028040339000085776
409[y]  no correction: 0.01169326527205602, correction:0.00036652777810589423
409[z]  no correction: 0.029886973511009877, correction:0.0007977217138609785
409[Vx]  no correction: 0.010705148118499512, correction:0.0004845653377526722
409[Vy]  no correction: 0.013922984334155323, correction:0.0005549264536912809
409[Vz]  no correction: 0.018425386659938234, correction:0.0006291482203397009


  1%|▊                                                                              | 1/100 [04:03<6:42:29, 243.94s/it]

411[x]  no correction: 0.03700791298711481, correction:0.01258217735638395
411[y]  no correction: 0.0180379351391425, correction:0.0022574978982897393
411[z]  no correction: 0.019204236601963957, correction:0.0036060996788831795
411[Vx]  no correction: 0.025439356982449878, correction:0.01391566006757299
411[Vy]  no correction: 0.012295113619725129, correction:0.004560439462685319
411[Vz]  no correction: 0.01744512354718017, correction:0.006407632202669285


  2%|█▌                                                                             | 2/100 [05:42<5:27:14, 200.35s/it]

...

 98%|████████████████████████████████████████████████████████████████████████████▍ | 98/100 [8:47:26<05:25, 162.66s/it]

596[x]  no correction: 0.03834668187083804, correction:0.013723502138803592
596[y]  no correction: 0.006510563373095324, correction:0.0009632990244615431
596[z]  no correction: 0.006935797954444145, correction:0.0023455558061840722
596[Vx]  no correction: 0.01656521055315475, correction:0.006368503788722379
596[Vy]  no correction: 0.005311120029185384, correction:0.002495334580074642
596[Vz]  no correction: 0.007580362160708127, correction:0.004461611979657295


 99%|█████████████████████████████████████████████████████████████████████████████▏| 99/100 [8:49:13<02:26, 146.04s/it]

597[x]  no correction: 0.0102068283760401, correction:0.0033465147514855644
597[y]  no correction: 0.021134070883808406, correction:0.005893589203534436
597[z]  no correction: 0.05646959030309932, correction:0.019642643196112714
597[Vx]  no correction: 0.005238040701819942, correction:0.0023720455483326824
597[Vy]  no correction: 0.02754174141454834, correction:0.01811000483524426
597[Vz]  no correction: 0.022383043297277926, correction:0.002767024353609174


100%|█████████████████████████████████████████████████████████████████████████████| 100/100 [8:50:38<00:00, 127.46s/it]


In [213]:
res.head()

,epoch,t,sat_id,x,y,z,Vx,Vy,Vz
id,,,,,,,,,
825619,1.391214e+18,31.010948,409,3920.707409,-5758.835921,25992.400795,2.692933,-3.154077,-0.827579
825620,1.391216e+18,31.040968,409,10495.538787,-13382.600634,22176.554605,2.328510,-2.669669,-2.058324
825621,1.391219e+18,31.070988,409,15814.667648,-19402.955486,15655.888817,1.750063,-1.948448,-2.899419
825622,1.391222e+18,31.101008,409,19512.385532,-23432.425437,7493.746745,1.098717,-1.158357,-3.332581
825623,1.391224e+18,31.131029,409,21534.491564,-25448.557777,-1337.682593,0.469478,-0.408800,-3.431058


In [16]:
res.to_csv('results_200_to_299.csv')

In [228]:
res1 = pd.read_csv('data/result_0_to_200.csv', index_col = 'id')
res1.head()

,epoch,t,sat_id,x,y,z,Vx,Vy,Vz
id,,,,,,,,,
3927,1.391213e+18,31.001217,1,-24616.438318,-10911.810034,6820.958757,3.083805,-1.304549,1.158497
3928,1.391214e+18,31.015938,1,-20565.072873,-12384.815035,7997.737262,3.778842,-0.994761,0.974115
3929,1.391215e+18,31.030658,1,-15631.615452,-13378.928776,8708.489841,4.494128,-0.539214,0.681806
3930,1.391217e+18,31.045378,1,-9832.429053,-13654.515179,8630.509884,5.063693,0.158111,0.208387
3931,1.391218e+18,31.060099,1,-3350.309837,-12793.435846,7249.772047,4.951922,1.293529,-0.592050


In [229]:
res2 = pd.read_csv('data/results_372_587_523_473_514.csv', index_col = 'id')
res2.tail()

,epoch,t,sat_id,x,y,z,Vx,Vy,Vz
id,,,,,,,,,
1054985,1.393626e+18,58.934400,514,233522.902580,-7526.987750,-3048.673310,0.0,-0.759704,-1.731971
1054986,1.393628e+18,58.949816,514,259115.628428,-1699.643703,3961.245890,0.0,3.536526,-1.942998
1054987,1.393629e+18,58.965232,514,284883.473292,5106.669098,8756.776852,0.0,5.285911,-1.790300
1054988,1.393630e+18,58.980648,514,309707.794495,9972.299398,10616.924199,0.0,4.727923,-0.670378
1054989,1.393632e+18,58.996063,514,334125.708799,12099.936750,11285.500085,0.0,2.230914,1.714616


In [231]:
df = res.append(res1)
# df = df.append(res2)

In [232]:
df.shape

(283983, 9)

In [233]:
test_data.shape

(284071, 9)

In [234]:
set(test_data.sat_id.values).difference(set(df.sat_id.values))

{301}

In [235]:
sat_301_data = transf_df[transf_df.sat_id ==301]
sat_301_data = sat_301_data[pd.isna(sat_301_data.x)]

In [236]:
sat_301_data = sat_301_data.drop(columns=['x','y','z','Vx','Vy','Vz'])
sat_301_data.columns = ['epoch','t','sat_id','x','y','z','Vx','Vy','Vz']
sat_301_data.head()

,epoch,t,sat_id,x,y,z,Vx,Vy,Vz
id,,,,,,,,,
589202,1.391235e+18,31.257923,301,-52592.999298,-60074.102890,-139573.635865,-1.323422,0.071619,0.418158
589203,1.391262e+18,31.573659,301,-86473.537938,-54655.422454,-158762.920979,-1.185425,0.208074,0.443904
589204,1.391290e+18,31.889396,301,-116576.927142,-47126.408813,-171085.163335,-1.049094,0.282414,0.469019
589205,1.391317e+18,32.205133,301,-143117.229722,-38203.789612,-178168.278315,-0.921780,0.330740,0.493551
589206,1.391344e+18,32.520869,301,-166305.946230,-28399.763252,-181059.157342,-0.801008,0.361295,0.517539


In [237]:
df = df.append(sat_301_data)
df.shape

(284071, 9)

In [238]:
sat_26_data = transf_df[transf_df.sat_id ==26]
sat_26_data = sat_26_data[pd.isna(sat_26_data.x)]
sat_26_data = sat_26_data.drop(columns=['x','y','z','Vx','Vy','Vz'])
sat_26_data.columns = ['epoch','t','sat_id','x','y','z','Vx','Vy','Vz']
sat_26_data.head()

,epoch,t,sat_id,x,y,z,Vx,Vy,Vz
id,,,,,,,,,
50725,1.391232e+18,31.221753,26,148699.986678,-15261.348146,-170190.755733,0.375843,0.721510,-0.262729
50726,1.391253e+18,31.465673,26,151614.711956,594.218047,-171985.237039,0.263672,0.690406,-0.131890
50727,1.391274e+18,31.709593,26,152288.113678,16428.611911,-171184.891073,0.156594,0.648208,-0.008761
50728,1.391295e+18,31.953512,26,150744.241316,32005.876510,-167885.541702,0.052642,0.593440,0.109114
50729,1.391316e+18,32.197432,26,146971.313778,47111.141442,-162141.599215,-0.049894,0.523472,0.223752


In [239]:
df.loc[sat_26_data.index, :] = sat_26_data.loc[:,:]

In [240]:
problem_index = df[df.sat_id.isin([372,523,587,473,514])].index.values
problem_index

array([959698, 959699, 959700, ..., 758375, 758376, 758377], dtype=int64)

In [133]:
# for sat_id in [252,372,523,587]:
#     id_data = transf_df[transf_df.sat_id ==sat_id]
#     id_data = id_data[pd.isna(id_data.x)]
#     id_data = id_data.drop(columns=['x','y','z','Vx','Vy','Vz'])
#     id_data.columns = ['epoch','t','sat_id','x','y','z','Vx','Vy','Vz']
#     df1 = df1.append(id_data)
# df1.shape

(284071, 9)

In [241]:
sub = pd.read_csv('data/submission.csv', index_col = 'id')

In [242]:
sub.head()

,x,y,z,Vx,Vy,Vz
id,,,,,,
3927,-13366.891347,-14236.753503,6386.774555,4.333815,-0.692764,0.810774
3928,-7370.434039,-14498.771520,7130.411325,5.077413,0.360609,0.313402
3929,-572.068654,-13065.289498,7033.794876,5.519106,2.012830,-0.539412
3930,6208.945257,-9076.852425,5548.296900,4.849212,4.338955,-1.869600
3931,10768.200284,-2199.706707,2272.014862,1.940505,6.192887,-3.167724


In [243]:
sumission_index = sub.index
df = df.loc[sumission_index,:]
# ignore problem data
df.loc[problem_index,:] = res2.loc[problem_index,:]
df.head()

,epoch,t,sat_id,x,y,z,Vx,Vy,Vz
id,,,,,,,,,
3927,1.391213e+18,31.001217,1,-24616.438318,-10911.810034,6820.958757,3.083805,-1.304549,1.158497
3928,1.391214e+18,31.015938,1,-20565.072873,-12384.815035,7997.737262,3.778842,-0.994761,0.974115
3929,1.391215e+18,31.030658,1,-15631.615452,-13378.928776,8708.489841,4.494128,-0.539214,0.681806
3930,1.391217e+18,31.045378,1,-9832.429053,-13654.515179,8630.509884,5.063693,0.158111,0.208387
3931,1.391218e+18,31.060099,1,-3350.309837,-12793.435846,7249.772047,4.951922,1.293529,-0.592050


In [244]:
df = df.drop(columns = ['epoch','t','sat_id'])

In [284]:
df.to_csv('submission_8.csv')